# Parse g_cons_raw

In [1]:
import os, re
import pandas as pd
import numpy as np

In [22]:
source = pd.read_csv('results_predictions_SP_input.txt', sep='\t', header=None)
source.columns = ['line','ref','raw','prediction']
source.index = source.line
source

,line,ref,raw,prediction
line,,,,
0,0,Genesis 1 1,BR>CJT,B-R>CJT/
1,1,Genesis 1 1,BR>,BR>[/
2,2,Genesis 1 1,>LHJM,>LH(J(M/JM
3,3,Genesis 1 1,>T,>T
4,4,Genesis 1 1,HCMJM,H-CMJ(M/(JM
...,...,...,...,...
20875,20875,Genesis 50 26,WJXNVW,Wn-J!XNV[W
20876,20876,Genesis 50 26,>TW,>T+W
20877,20877,Genesis 50 26,WJWCM,Wn-J!(H](J&WCM[


In [19]:
def g_cons_trailer(w):

    retain_special = '-<>'
    w = re.sub(r'\([\w]|\([><]','', w) #Remove suffixes, e.g. (H
    w = re.sub('[^A-Z'+retain_special+']', '', w) #Retain only capital letters plus special letters
    
    return '-'.join(w.split('-')) #split string according to hyphen.

def check_corrections(df):
    
    df['test'] = [re.sub('-', '', g_cons_trailer(w)) for w in list(df.parsed)]
    mismatches = df[df.raw != df.test]
    
    if len(mismatches) > 0:
        return mismatches
    else:
        return "All mismatches corrected"

In [20]:
g_cons_trailer('W->HL/JM~H')

'W->HLJMH'

#### Make predictions and test parsing:

In [23]:
source['parsed'] = [g_cons_trailer(w) for w in list(source.prediction)]
source['test'] = [re.sub('-', '', g_cons_trailer(w)) for w in list(source.prediction)]
source

,line,ref,raw,prediction,parsed,test
line,,,,,,
0,0,Genesis 1 1,BR>CJT,B-R>CJT/,B-R>CJT,BR>CJT
1,1,Genesis 1 1,BR>,BR>[/,BR>,BR>
2,2,Genesis 1 1,>LHJM,>LH(J(M/JM,>LHJM,>LHJM
3,3,Genesis 1 1,>T,>T,>T,>T
4,4,Genesis 1 1,HCMJM,H-CMJ(M/(JM,H-CMJM,HCMJM
...,...,...,...,...,...,...
20875,20875,Genesis 50 26,WJXNVW,Wn-J!XNV[W,W-JXNVW,WJXNVW
20876,20876,Genesis 50 26,>TW,>T+W,>TW,>TW
20877,20877,Genesis 50 26,WJWCM,Wn-J!(H](J&WCM[,W-JWCM,WJWCM


In [24]:
mismatches = source[source.raw != source.test]
mismatches

,line,ref,raw,prediction,parsed,test
line,,,,,,
174,174,Genesis 1 14,L>TWT,L->T==+W,L->TW,L>TW
179,179,Genesis 1 15,LM>WRWT,L-!M>&WR[/W,L-M>WRW,LM>WRW
442,442,Genesis 2 2,WJKL,Wn-J!(HLK[,W-JLK,WJLK
607,607,Genesis 2 14,HCLJCJ,H]CL&JC[,HCLJC,HCLJC
614,614,Genesis 2 14,HRBJ<J,H]RB&J<[,HRBJ<,HRBJ<
...,...,...,...,...,...,...
20347,20347,Genesis 49 23,WJMRRHW,Wn-J!MRRR[+HW,W-JMRRRHW,WJMRRRHW
20351,20351,Genesis 49 23,XYJM,XY/,XY,XY
20399,20399,Genesis 49 27,BNJMJM,BN/JM,BNJM,BNJM


#### Import and check corrections:

In [33]:
corrections = pd.read_excel('./data/corrections.xlsx')
source['parsed'] = list(corrections['g_cons_raw'])

In [39]:
corrections

,Unnamed: 0,line,ref,raw,C_disambig,g_cons_raw,lex
0,0,0,Genesis 1 1,BR>CJT,BR>CJT,B-R>CJT,B-R>CJT/
1,1,1,Genesis 1 1,BR>,BR>,BR>,BR>[
2,2,2,Genesis 1 1,>LHJM,>LHJM,>LHJM,>LHJM/
3,3,3,Genesis 1 1,>T,>T,>T,>T
4,4,4,Genesis 1 1,HCMJM,HCMJM,H-CMJM,H-CMJM/
...,...,...,...,...,...,...,...
20858,20858,20875,Genesis 50 26,WJXNVW,WJXNVW,W-JXNVW,W-XNV[
20859,20859,20876,Genesis 50 26,>TW,>TW,>TW,>T
20860,20860,20877,Genesis 50 26,WJWCM,WJWCM,W-JWCM,W-JCM[
20861,20861,20878,Genesis 50 26,B>RN,B>RN,B->RN,B->RN/


In [35]:
check_corrections(source)

'All mismatches corrected'

In [36]:
source['g_cons_raw'] = source['parsed']
source = source[['ref','raw','g_cons_raw']]

## Disambiguate C and F

#### Make dataset:

In [2]:
from tf.app import use
A = use('DT-UCPH/sp:hot', hoist=globals())

**Locating corpus resources ...**

rate limit is 5000 requests per hour, with 5000 left for this hour
	connecting to online GitHub repo DT-UCPH/sp ... connected
	app/__init__.py...downloaded
	app/app.py...downloaded
	app/config.yaml...downloaded
	app/static...directory
		app/static/logo.png...downloaded
	OK


The requested data is not available offline
	C:/Users/Christian/text-fabric-data/github/DT-UCPH/sp/tf/2.0 not found
rate limit is 5000 requests per hour, with 4989 left for this hour
	connecting to online GitHub repo DT-UCPH/sp ... connected
	cannot find releases
	cannot find releases
	tf/2.0/.tf...directory
		tf/2.0/.tf/3...directory
			tf/2.0/.tf/3/__boundary__.tfx...excluded
			tf/2.0/.tf/3/__characters__.tfx...excluded
			tf/2.0/.tf/3/__levDown__.tfx...excluded
			tf/2.0/.tf/3/__levUp__.tfx...excluded
			tf/2.0/.tf/3/__levels__.tfx...excluded
			tf/2.0/.tf/3/__order__.tfx...excluded
			tf/2.0/.tf/3/__rank__.tfx...excluded
			tf/2.0/.tf/3/__sections__.tfx...excluded
			tf/2.0/.tf/3/book.tfx...excluded
			tf/2.0/.tf/3/chapter.tfx...excluded
			tf/2.0/.tf/3/g_cons.tfx...excluded
			tf/2.0/.tf/3/g_lex.tfx...excluded
			tf/2.0/.tf/3/g_nme.tfx...excluded
			tf/2.0/.tf/3/g_pfm.tfx...excluded
			tf/2.0/.tf/3/g_prs.tfx...excluded
			tf/2.0/.tf/3/g_uvf.tfx...excluded
			tf/2.0/.tf/3/g_vbe.tf

   |     0.28s T otype                from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |     2.40s T oslots               from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |     0.01s T chapter              from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |     0.03s T verse                from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |     1.07s T sign                 from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |     0.01s T book                 from ~/text-fabric-data/github/DT-UCPH/sp/tf/2.0
   |      |     0.06s C __levels__           from otype, oslots, otext
   |      |     2.27s C __order__            from otype, oslots, __levels__
   |      |     0.11s C __rank__             from otype, __order__
   |      |     2.87s C __levUp__            from otype, oslots, __rank__
   |      |     1.84s C __levDown__          from otype, __levUp__, __rank__
   |      |     0.03s C __characters__       from otext
   |      |     1.64s C __boundary__         from otype, 

Name,# of nodes,# slots/node,% coverage
book,5,49966.20,100
chapter,187,1335.99,100
verse,5841,42.77,100
word,116677,2.14,100
sign,249831,1.00,100


In [10]:
words = [w for w in F.otype.s('word') if T.sectionFromNode(w)[0] in {'Exodus','Leviticus','Numbers','Deuteronomy'}]

g_cons_raw = [F.g_cons_raw.v(w) for w in words]
book = [T.sectionFromNode(w)[0] for w in words]
chapter = [T.sectionFromNode(w)[1] for w in words]
verse = [T.sectionFromNode(w)[2] for w in words]

df = pd.DataFrame([book, chapter, verse, g_cons_raw]).T

In [11]:
df

,0,1,2,3
0,Exodus,1,1,W
1,Exodus,1,1,>LH
2,Exodus,1,1,CMWT
3,Exodus,1,1,BNJ
4,Exodus,1,1,JCR>L
...,...,...,...,...
87626,Deuteronomy,34,12,MCH
87627,Deuteronomy,34,12,L
87628,Deuteronomy,34,12,<JNJ
87629,Deuteronomy,34,12,KL


In [13]:
df.to_csv('../data/SP_input_disambiguate_C_Exod_Deut.txt', sep='\t', header=None, index=False)

Merge corrections:

In [40]:
from tf.app import use
A = use('app:./app', hoist=globals())

In [41]:
for v in F.otype.s('verse'):
    bo, ch, ve = T.sectionFromNode(v)
    ref = f"{bo} {ch} {ve}"
    lenA = len(corrections[corrections.ref == ref])
    lenB = len(source[source.ref == ref])
    
    if lenA != lenB:
        print(ref)

In [42]:
source['C_disambig'] = list(corrections.C_disambig)
source

,ref,raw,g_cons_raw,book,chapter,verse,C_disambig
line,,,,,,,
0,Genesis 1 1,BR>CJT,B-R>CJT,Genesis,1,1,BR>CJT
1,Genesis 1 1,BR>,BR>,Genesis,1,1,BR>
2,Genesis 1 1,>LHJM,>LHJM,Genesis,1,1,>LHJM
3,Genesis 1 1,>T,>T,Genesis,1,1,>T
4,Genesis 1 1,HCMJM,H-CMJM,Genesis,1,1,HCMJM
...,...,...,...,...,...,...,...
20875,Genesis 50 26,WJXNVW,W-JXNVW,Genesis,50,26,WJXNVW
20876,Genesis 50 26,>TW,>TW,Genesis,50,26,>TW
20877,Genesis 50 26,WJWCM,W-JWCM,Genesis,50,26,WJWCM


If the disambiguated units do not contain both F and C, the disambiguation can safely be imposed upon g_cons_raw to create a new feature: g_cons

In [43]:
f'Number of cases with both F and C: {len([p for p in source.C_disambig if "F" in p and "C" in p])}'

'Number of cases with both F and C: 4'

In [44]:
g_cons = []

for n, row in source.iterrows():
    if 'F' in row['C_disambig']:
        g_cons.append(re.sub('C','F', row['g_cons_raw']))
        if 'F' in row['C_disambig'] and 'C' in row['C_disambig']:
            print(f"{n}\t{row['C_disambig']}")
    else:
        g_cons.append(row['g_cons_raw'])
        
source['g_cons'] = g_cons

11456	JCFKR
14082	WJCFKR
19004	JCFKR
20274	JCFKR


In [45]:
source = source[['ref','raw','g_cons_raw','g_cons']]

In [46]:
source[source.ref == 'Genesis 36 25']

,ref,raw,g_cons_raw,g_cons
line,,,,
14454,Genesis 36 25,W>LH,W->LH,W->LH
14455,Genesis 36 25,BNJ,BNJ,BNJ
14456,Genesis 36 25,<NH,<NH,<NH
14457,Genesis 36 25,DJCWN,DJCWN,DJCWN
14458,Genesis 36 25,W>HLJBMH,W->HLJBMH,W->HLJBMH
14459,Genesis 36 25,BT,BT,BT
14460,Genesis 36 25,<NH,<NH,<NH


## Export

In [47]:
source.to_csv('./data/g_cons.txt', sep='\t')